In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re
import requests
import numpy as np
import spacy
import time
import datetime
nlp = spacy.load('en')

In [45]:
def match_num(text):
    number_words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety', 'hundred','hundreds', 'thousand','thousands']
    text = text.replace('COVID-19',' ')
    has_digit = re.search('\d+',text)
    if not has_digit:
        for word in number_words:
            pattern = ' ' + word + ' '
            if re.search(pattern,text):
                has_digit = True
                break
    return has_digit
    

In [46]:
def get_links():
    resp = urllib.request.urlopen("https://globalnews.ca/?s=alberta+coronavirus")
    soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    link_list = []
    pattern = 'https://globalnews.ca/news/\d+/.*/$'
    for link in soup.find_all('a', href=True):
        if re.match(pattern,link['href']):
            link_list.append(link['href'])
        link_set = set(link_list)
    unique_link_list = list(link_set)
    return unique_link_list

In [49]:
def show_sents(unique_link_list):
    for link in unique_link_list:
        time.sleep(1)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html.parser')
        # Show date
        post_time =soup_article.find_all('div', class_='c-byline__date')
        date =[]
        for t in post_time:
            date.append(t.get_text())
        # Show number related phrase
        body = soup_article.find_all('div')#, class_='l-article__part'
        x = body[0].find_all('p')
        news_contents = ''
        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(x)):
            paragraph = x[p].get_text()
            if not 'read more' in  paragraph.lower():
                list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)
        spacy_doc = nlp(final_article)
        related_text = ""
        for sent in spacy_doc.sents:
            if 'case' in sent.text and 'new' in sent.text:
                if match_num(sent.text):
                    clean_text = sent.text.replace('\n',' ')
                    clean_text = clean_text.replace('\t','')
                    related_text =related_text + clean_text + '\n'
        if related_text:
            print('Title: ' + soup_article.title.get_text())
            print(date)
            print('Phrases: ')
            print(related_text)

In [ ]:
prev_set = set()
while True:
    unique_link_list = get_links()
    new_link = list(set(unique_link_list).difference(prev_set))
    if new_link:
        print(datetime.datetime.now())
        show_sents(new_link)
    prev_set = set(unique_link_list)
    time.sleep(3600)

2020-03-17 23:04:24.584429
Title: Confirmed coronavirus cases in Alberta reach 39; 2 patients in intensive care  | Globalnews.ca
['Posted March 14, 2020 6:20 pm', 'Updated March 15, 2020 7:23 pm']
Phrases: 
Alberta Health confirmed Saturday afternoon that there are 10 new cases of COVID-19 in the province, bringing the total to 39.
Six of the new cases are in the Calgary zone and four are in the Edmonton zone, Alberta Health said.
Two of the new cases — one in the Edmonton zone and one in the Calgary zone — have been admitted into intensive care, according to Alberta Health.
Alberta Health said eight of the new cases are related to international travel or to a previously confirmed case, adding that the other two cases are being investigated.

Title: Coronavirus cases in Alberta and B.C. are linked. What does that mean for further spread?  | Globalnews.ca
['Posted March 8, 2020 4:38 pm', 'Updated March 8, 2020 4:44 pm']
Phrases: 
B.C.’s Ministry of Health confirmed Sunday that the B.C. 